In [3]:
import pandas as pd
import os
from time import ctime
from glob import glob

import warnings
warnings.filterwarnings('ignore')

In [4]:
os.getcwd()

'F:\\Documents\\diplom'

In [5]:
os.chdir(r'F://Downloads/')

Данные с этого соревнования
https://www.kaggle.com/c/acquire-valued-shoppers-challenge/overview

Данные содержат информацию о транзакциях клиентов и составе корзины покупок.
___

| Признак | Описание 
| -------- | -------- 
| id     |Id клиента
| chain | Код сети магазинов 
|market| Код географического региона
|dept | Код категории товаров (вода)
|category | Код продуктовой категории (газированная вода)
|company | Код компании, продающей товар
|brand | Код бренда
|date | Дата транзакции
|productsize | вес, количество продукта
|productmeasure | Единицы измерения
|purchasequantity | Число купленные единиц
|purchaseamount | Сумма

* 35 млн строк (2.33 ГБ)
* 27 258 уникальных клиентов
* Период: с марта 2012 по май 2013 


---

In [30]:
print('%d уникальной торговой сети' %  pd.unique(data.chain).shape[0])
print('%d уникальных категорий товаров' %  pd.unique(data.dept).shape[0])
print('%d уникальной подкатегории товаров' %  pd.unique(data.category).shape[0])
print('%d уникальных компаний, продающих товары' %  pd.unique(data.company).shape[0])
print('%d уникальных брендов' %  pd.unique(data.brand).shape[0])

80 уникальной торговой сети
82 уникальных категорий товаров
821 уникальной подкатегории товаров
15713 уникальных компаний, продающих товары
17040 уникальных брендов


In [4]:
# Исходные данные
pd.read_csv('transactions.csv', nrows=10)

,id,chain,dept,category,company,brand,date,productsize,productmeasure,purchasequantity,purchaseamount
0,86246,205,7,707,1078778070,12564,2012-03-02,12,OZ,1,7.59
1,86246,205,63,6319,107654575,17876,2012-03-02,64,OZ,1,1.59
2,86246,205,97,9753,1022027929,0,2012-03-02,1,CT,1,5.99
3,86246,205,25,2509,107996777,31373,2012-03-02,16,OZ,1,1.99
4,86246,205,55,5555,107684070,32094,2012-03-02,16,OZ,2,10.38
5,86246,205,97,9753,1021015020,0,2012-03-02,1,CT,1,7.80
6,86246,205,99,9909,104538848,15343,2012-03-02,16,OZ,1,2.49
7,86246,205,59,5907,102900020,2012,2012-03-02,16,OZ,1,1.39
8,86246,205,9,921,101128414,9209,2012-03-02,4,OZ,2,1.50
9,86246,205,73,7344,1068142161,20285,2012-03-02,8,CT,1,5.79


In [6]:
example_sample = pd.read_csv('transactions.csv', nrows=1_000_000)
# example_sample.to_csv('transactions_sample.csv')
example_sample.id.unique().shape # 1050 уникальных клиентов

(1050,)

---

С данными в исходном виде работать сложно.

Преобразуем их в более удобный формат т.е. сопоставим каждой дате группу клиентов с агрегированными характеристиками их покупок

---

In [9]:
# Вход - датасет такой то

numberOfSegments = 6
lenghtOfSegment = 25_000_000 #30_000_000
skiprows_list = [range(1,lenghtOfSegment*x+1) for x in range(0,numberOfSegments)]
# Чтение данных и перевод в to_datetime тут должно быть

In [10]:
print(ctime() + '\tПроцесс пошел.')
for idx, x in enumerate(skiprows_list):
    # импорт данных
    data = pd.read_csv('transactions.csv', nrows=lenghtOfSegment, skiprows=x)
    data.date = pd.to_datetime(data.date)
    print(ctime() +'\t   Импорт данных')    
    # Считаем среднюю стоимость товара в корзине покупателя
    data1 = (
    data
    .groupby(by=['date', 'id'], as_index=False)['purchaseamount']
    .mean()
    .rename(columns={'purchaseamount':'mean_sales'})
    )
    print(ctime() +'\t   Вычисление ср.стоимости товара в корзине покупателя')
    # Считаем сумму корзины
    data2 = (
    data
    .groupby(by=['date', 'id'], as_index=False)[['purchaseamount']]
    .sum()
    .rename(columns={'purchaseamount':'sales'})
    )
    print(ctime() +'\t   Вычисление суммы корзины')
    data3 = data1.join(data2[['sales']], rsuffix='_') # Соединяем в единую таблицу
    print(ctime() +'\t   Join data1 and data2')
    # Вычисляем временные признаки
    
    # ОЧЕНЬ ДОЛГО ВЫЧИСЛЯЕТ
#     data3['month_name'] = data.date.apply(lambda x: x.month_name())
#     print(ctime() +'\tMONTH NAME')
#     data3['day_of_week_name'] = data.date.apply(lambda x: x.day_name())
#     print(ctime() +'\tDAY NAME')
#     data3['month'] = data.date.apply(lambda x: x.month) # 1 - january
#     print(ctime() +'\t   вычисление столбца month')
#     data3['day_of_week'] = data.date.apply(lambda x: x.dayofweek) # 0 - monday]
#     print(ctime() +'\t   вычисление столбца dayOfWeek')
#     data3['day'] = data.date.apply(lambda x: x.day)
#     print(ctime() +'\t   вычисление столбаца day')
#     data3['year'] = data.date.apply(lambda x: x.year)  
#     print(ctime() +'\t   вычисление стообца year')
    
    # Сохраняем
    data3_name = 'data_prep_{0}.csv'.format(idx)     
    print(ctime() + '\tПредобработка данных завершена, сохраняем ' + data3_name + ' ...')
    data3.to_csv(data3_name, index = False)
    print(ctime() + '\tСохранили ' + data3_name)
    
print(ctime() +'\tЭтап 1. Преобразование исходного массива успешно завершен.')

    # Склеиваем всё в единый большой массив

files = glob(data3_name[:-5] + '*') # Получили имена файлов
data = pd.read_csv(files.pop(0))
while files:
    data2 = pd.read_csv(files.pop(0))
    data = pd.concat([data, data2], ignore_index=True)
print(ctime() + '\tЭтап 2. Склеивание преобразованных массивов завершено.')

# Тут образуются дубликаты на разрывах, их нужно как то обработать!

dictonary = dict()
def getNewId(key_id):
    '''
    Присвоить id новые порядковые номера
    '''
    if key_id not in dictonary:
        if not dictonary: # Если словарь пустой  
            dictonary[key_id] = 1
        else:
            dictonary[key_id] = max(dictonary.values()) + 1
    
    return dictonary[key_id]

data.id = data.id.apply(lambda x: getNewId(x)) # Присвоить новые id
print(ctime() + '\tПосетителям назначены новые ID.')

data = data[data.date < '2013-05-01'] # Оставляем нужный период времени
print(ctime() + '\tОтфильтрован временной промежуток.')

data = data.sort_values(by=['date', 'id']) # Сортируем по дате и ID

data.to_csv('transactions_prep.csv', index=False)
print(ctime() + '\tЭтап 3. Исходные данные полностью преобразованы и сохранены в "transactions_prep.csv".')



Tue Oct 13 22:20:59 2020	Процесс пошел.
Tue Oct 13 22:21:24 2020	   Импорт данных
Tue Oct 13 22:21:26 2020	   Вычисление ср.стоимости товара в корзине покупателя
Tue Oct 13 22:21:28 2020	   Вычисление суммы корзины
Tue Oct 13 22:21:29 2020	   Join data1 and data2
Tue Oct 13 22:21:29 2020	Предобработка данных завершена, сохраняем data_prep_0.csv ...
Tue Oct 13 22:21:51 2020	Сохранили data_prep_0.csv
Tue Oct 13 22:22:29 2020	   Импорт данных
Tue Oct 13 22:22:31 2020	   Вычисление ср.стоимости товара в корзине покупателя
Tue Oct 13 22:22:34 2020	   Вычисление суммы корзины
Tue Oct 13 22:22:34 2020	   Join data1 and data2
Tue Oct 13 22:22:34 2020	Предобработка данных завершена, сохраняем data_prep_1.csv ...
Tue Oct 13 22:22:50 2020	Сохранили data_prep_1.csv
Tue Oct 13 22:23:34 2020	   Импорт данных
Tue Oct 13 22:23:36 2020	   Вычисление ср.стоимости товара в корзине покупателя
Tue Oct 13 22:23:38 2020	   Вычисление суммы корзины
Tue Oct 13 22:23:38 2020	   Join data1 and data2
Tue Oct 13 2

In [70]:
# Около часа будет считать

In [11]:
# load data
transactions = pd.read_csv('transactions_prep.csv') #global
transactions['date'] = pd.to_datetime(transactions['date'])

In [12]:
transactions[transactions.date <= '20120427'].id.unique().shape

(114673,)